In [72]:
from netCDF4 import Dataset
import numpy as np
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

import traceback
def mergeImagesFile(fileName, p):
    data = []
    errors = []
    try:
        ds = Dataset(fileName)
        coord = ds.groups['coordenadas']
        lats = coord['latitude'][:].data
        lons = coord['longitude'][:].data

        x, y = findStationCoords(lons, lats, p['coordLon'], p['coordLat'])

        margen = int(p['margen'] / 2)
        for i in range(len(p['tiempos'])-1,-1,-1):
            canalImages = []
            for c in p['canales']:
                cmis = ds.groups[f'{c}-{i}']
                cmi = cmis.variables['CMI'][:].data.astype(np.uint16)

                if p['dibujar'] and p['canalDibujar'] == c and i == 0:
                    try:
                        dibujarMapa(lons, lats, cmi, mersh=0, point=[p['coordLon'], p['coordLat']])
                    except:
                        errors.append('No se pudo dibujar la imagen')
                        pass

                cropedImg = cmi[y - margen:y + margen, x - margen:x + margen]
                canalImages.append(cropedImg)

            data.append(np.dstack(canalImages))

        data = np.stack(data, axis=0)

        ds.close()


    except Exception as e:
        print(str(e))
        ds.close()
        traceback.print_exc()
        return [0], errors.append(f'No se pudo unir las imagenes: Tiempo: {i}- Canal: {c}')

    return data, errors

def findStationCoords(lons, lats, cordX, cordY):
    x = getPosMap(cordX, lons)
    y = getPosMap(cordY, lats)
    return x, y
def getPosMap(x, array):
    pos = -1
    for i in range(len(array)):
        if abs(array[i] - x) <= 0.01:
            pos = i

    return pos


In [58]:
import tensorflow as tf

# Ruta del archivo HDF5
model_path = r'C:\Users\Shounen\Desktop\Ciclo XI\Tesis 2\FinalTesis\Tesis2-DiegoParedes\Archivos\Resultados\Clasificacion\CONV3D-V20241021\Clasificacion-20241027_12\Model_195_CONV3D_clase_20241028_213953.hdf5'
model_path = r'C:\Users\Shounen\Desktop\Ciclo XI\Tesis 2\Test\SistemaQC\models\Model_02_CONV3D_clase_20240927_152142.hdf5'
# Cargar el modelo
model = tf.keras.models.load_model(model_path)

# Ver el resumen del modelo para verificar que se ha cargado correctamente
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 6, 8, 8, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 rescaling_5 (Rescaling)        (None, 6, 8, 8, 3)   0           ['input_21[0][0]']               
                                                                                                  
 conv3d_23 (Conv3D)             (None, 6, 8, 8, 32)  2624        ['rescaling_5[0][0]']            
                                                                                                  
 conv3d_24 (Conv3D)             (None, 6, 8, 8, 16)  13840       ['conv3d_23[0][0]']        

In [59]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Shounen\Desktop\Ciclo XI\Tesis 2\SistemaQC-Precipitaciones\static\estaciones.csv')

In [60]:
df.set_index('CODE').to_dict(orient='index')

{'X47E0D438': {'Unnamed: 0': 0,
  'ESTACION': 'ALAMOR',
  'LON': -80.39788,
  'LAT': -4.48047,
  'ALT': 116.0},
 'X47E09732': {'Unnamed: 0': 1,
  'ESTACION': 'LA ARDILLA',
  'LON': -80.39014,
  'LAT': -4.48956,
  'ALT': 116.0},
 'X472606FA': {'Unnamed: 0': 2,
  'ESTACION': 'AYABACA',
  'LON': -79.71077,
  'LAT': -4.63776,
  'ALT': 2633.0},
 'X47E01126': {'Unnamed: 0': 3,
  'ESTACION': 'CABO INGA H',
  'LON': -80.39898,
  'LAT': -3.97972,
  'ALT': 143.0},
 'X472F00A6': {'Unnamed: 0': 4,
  'ESTACION': 'CABO INGA M',
  'LON': -80.40182,
  'LAT': -3.97594,
  'ALT': 228.0},
 'X472EF2D8': {'Unnamed: 0': 5,
  'ESTACION': 'CAMPAMENTO SEDE',
  'LON': -80.42964,
  'LAT': -3.55117,
  'ALT': 12.0},
 'X472F86B2': {'Unnamed: 0': 6,
  'ESTACION': 'CANCHAQUE',
  'LON': -79.60535,
  'LAT': -5.40058,
  'ALT': 1270.0},
 'X472F13D0': {'Unnamed: 0': 7,
  'ESTACION': 'CA?AVERAL',
  'LON': -80.65058,
  'LAT': -3.93911,
  'ALT': 144.0},
 'X472FB328': {'Unnamed: 0': 8,
  'ESTACION': 'CHALACO',
  'LON': -79.83,

In [87]:
import sklearn
sklearn.__version__


'1.0.2'

In [61]:
filename = r'C:\Users\Shounen\Desktop\Ciclo XI\Tesis 2\Test\SistemaQC\imagenes\2021-12-29-14-00.nc' # GOES 23
data = []
ds = Dataset(filename)
coord = ds.groups['coordenadas']
lats = coord['latitude'][:].data
lons = coord['longitude'][:].data


for i in range(6-1,-1,-1):
    canalImages = []
    for c in ["07", "08", "13"]:
        cmis = ds.groups[f'{c}-{i}']
        cmi = cmis.variables['CMI'][:].data.astype(np.uint16)
        canalImages.append(cmi)

    data.append(np.dstack(canalImages))

data = np.stack(data, axis=0)

ds.close()

FileNotFoundError: [Errno 2] No such file or directory: b'C:\\Users\\Shounen\\Desktop\\Ciclo XI\\Tesis 2\\Test\\SistemaQC\\imagenes\\2021-12-29-14-00.nc'

In [62]:
float('asd')

ValueError: could not convert string to float: 'asd'

In [63]:
import GOES
a = GOES.__version__

In [77]:

inputLayers = []
config = model.get_config()
for layer in config['layers']:
    if layer['class_name'] == 'InputLayer':
        inputLayers.append(layer['name'])
inputLayers        

['input_21', 'input_22', 'input_23', 'input_24']

In [79]:
from netCDF4 import Dataset
p =  {'port': 8080, 'umbral': 0.51, 'sizeMax': 1000000.0, 'save': True,
      'domain': [-88.0, -63.0, -25.0, 5.0], 'saveMax': 10000001,
      'dirModelos': 'C:\\Users\\Shounen\\Desktop\\Ciclo XI\\Tesis 2\\SistemaQC/Modelos/', 'fecha': '2021-12-20-06-00', 'dato': 40.2,
      'canales': ['07', '08', '13'], 'tiempos': ['00', '50', '40','30','20','10'], 'margen': 8, 'dibujar': False,
      'canalDibujar': '13', 'hard_save': False, 'coordLon': -75.9825, 'coordLat': -9.34694, 'codigo': 'X47E0E1A2', 'ID': 1726442740}

fileName = r'C:\Users\Shounen\Desktop\Ciclo XI\Tesis 2\SistemaQC\Imagenes\2022-02-01-07-00.nc'
imagen, errors = mergeImagesFile(fileName, p)
a = {
     inputLayers[0]: np.full((1, imagen.shape[0], imagen.shape[1],imagen.shape[2], imagen.shape[3]),imagen),
     inputLayers[1]: np.full((1,), 40.2),  # Precipitacion
     inputLayers[3]: np.full((1,), 644),  # Altura
     inputLayers[2]: np.full((1,), 16.46)  # Umbral
    }

max_data = 10

In [85]:
import pandas as pd
import pickle

from sklearn.preprocessing import MinMaxScaler
atrain = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Dataset/Clasificacion/Entrenamiento/ClaseV20242209_DAUS/CLASE_TrainDS_1.csv'
dfTrain = pd.read_csv(atrain)#, cols=[''])

scaler = MinMaxScaler()
if max_data:    
    print('max-data', max_data)
    precip_data_truncated = np.clip(dfTrain['dato'], 0, max_data)
    dfTrain['_dato'] = scaler.fit_transform(np.array(precip_data_truncated).reshape(-1, 1))
    
    with open('scaler_dato.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    
    
    precip_data_truncated = np.clip(a[inputLayers[1]], 0, max_data)
    a[inputLayers[1]] = scaler.transform(np.array(precip_data_truncated).reshape(-1, 1))


    dfTrain['_umb1'] = scaler.fit_transform(np.array(dfTrain['umb1']).reshape(-1, 1))
    a[inputLayers[2]] = scaler.transform(np.array(a[inputLayers[2]]).reshape(-1, 1))
    
    with open('scaler_umb.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    
    

    dfTrain['_altura'] = scaler.fit_transform(np.array(dfTrain['altura']).reshape(-1, 1))
    a[inputLayers[3]] = scaler.transform(np.array(a[inputLayers[3]]).reshape(-1, 1))

    with open('scaler_alt.pkl', 'wb') as f:
        pickle.dump(scaler, f)

max-data 10


In [83]:
max_data

10

In [81]:
model.predict(a)

array([[0.43736872]], dtype=float32)

In [7]:
data_layers = []
for layer in model.layers:
    intermediate_model = Model(inputs=model.input, outputs=layer.output)
    intermediate_output = intermediate_model.predict(a)
    data_layers.append(intermediate_output)
    print(f"Layer {layer.name}  Shape ({intermediate_output.shape}): activations max {np.max(intermediate_output)}, min {np.min(intermediate_output)}")


Layer input_22  Shape ((1, 6, 12, 12, 3)): activations max 28363.0, min 19444.0
Layer rescaling_6  Shape ((1, 6, 12, 12, 3)): activations max 0.4327850341796875, min 0.29669189453125
Layer conv3d_36  Shape ((1, 6, 12, 12, 32)): activations max 3.762014627456665, min -3.3437821865081787
Layer batch_normalization_36  Shape ((1, 6, 12, 12, 32)): activations max 2.929373264312744, min -4.151337623596191
Layer leaky_re_lu_36  Shape ((1, 6, 12, 12, 32)): activations max 2.929373264312744, min -0.04151337593793869
Layer conv3d_37  Shape ((1, 6, 12, 12, 16)): activations max 36.4212646484375, min -68.4900131225586
Layer batch_normalization_37  Shape ((1, 6, 12, 12, 16)): activations max 3.0112760066986084, min -3.059765577316284
Layer leaky_re_lu_37  Shape ((1, 6, 12, 12, 16)): activations max 3.0112760066986084, min -0.030597655102610588
Layer dropout_45  Shape ((1, 6, 12, 12, 16)): activations max 3.0112760066986084, min -0.030597655102610588
Layer conv3d_38  Shape ((1, 6, 12, 12, 32)): acti

In [8]:
data_layers[2].shape

(1, 6, 12, 12, 32)

In [9]:
import tensorflow as tf
import numpy as np

def get_grad_cam_3d(model, input_data, layer_name, class_idx):
    # Construir el modelo hasta la capa de interés
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(layer_name).output, model.output]
    )

    # Registrar gradientes con GradientTape
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(input_data)
        loss = predictions[:, class_idx]

    # Obtener gradientes de la salida con respecto a la capa convolucional
    grads = tape.gradient(loss, conv_outputs)

    # Pool de los gradientes (media a través de dimensiones espaciales)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2, 3))

    # Multiplicar cada canal por la importancia de los gradientes
    conv_outputs = conv_outputs[0]  # Eliminar la dimensión de batch
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    # Aplicar ReLU para eliminar los valores negativos
    heatmap = np.maximum(heatmap, 0)

    # Normalizar entre 0 y 1 para visualizarlo como mapa de calor
    heatmap /= np.max(heatmap) if np.max(heatmap) != 0 else 1
    
    return heatmap#.numpy()


In [10]:
import matplotlib.pyplot as plt

def show_grad_cam_heatmap_3d(heatmap, input_volume, slice_idx=None, axis=0):
    """
    Visualiza el heatmap proyectado a través de una de las dimensiones del volumen.
    
    Args:
    - heatmap: El heatmap generado por Grad-CAM.
    - input_volume: El volumen de entrada original (imágenes).
    - slice_idx: El índice de la rebanada específica para superponer el heatmap (opcional).
    - axis: El eje a lo largo del cual promediar el heatmap (0 = eje Z, 1 = eje Y, 2 = eje X).
    """
    if slice_idx is None:
        # Promediar el heatmap a lo largo del eje especificado para obtener una proyección 2D
        heatmap_2d = np.mean(heatmap, axis=axis)
    else:
        # Tomar una rebanada específica
        heatmap_2d = heatmap.take(indices=slice_idx, axis=axis)
    
    # Normalizar el heatmap entre 0 y 1 para visualizar
    heatmap_2d = np.maximum(heatmap_2d, 0)
    heatmap_2d /= np.max(heatmap_2d) if np.max(heatmap_2d) != 0 else 1

    # Visualizar el heatmap y opcionalmente el volumen de entrada
    plt.figure(figsize=(8, 8))

    # Si quieres mostrar una rebanada del volumen original
    if input_volume is not None and slice_idx is not None:
        input_slice = input_volume.take(indices=slice_idx, axis=axis)
        plt.subplot(1, 2, 1)
        plt.title(f'Input Slice (axis {axis}, slice {slice_idx})')
        plt.imshow(input_slice, cmap='gray')
    
    plt.subplot(1, 2, 2)
    plt.title(f'Heatmap Projection (axis {axis})')
    plt.imshow(heatmap_2d, cmap='jet')  # 'jet' colormap para ver mejor el heatmap
    plt.colorbar()
    plt.show()


In [11]:
plt.imshow(input_volume[1], cmap='gray')

NameError: name 'input_volume' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_grad_cam_heatmap_3d(heatmap, input_volume, axis=0):
    """
    Visualiza el input y el heatmap para múltiples slices en una cuadrícula.
    
    Args:
    - heatmap: El heatmap generado por Grad-CAM.
    - input_volume: El volumen de entrada original (imágenes).
    - axis: El eje a lo largo del cual obtener los slices (0 = eje Z, 1 = eje Y, 2 = eje X).
    """
    num_slices = 6  # Número de slices a mostrar (0 a 5)
    
    # Crear la figura con una cuadrícula de subplots
    fig, axs = plt.subplots(2 * num_slices, 2, figsize=(12, 4 * num_slices))
    
    for i in range(num_slices):
        # Tomar el slice específico del heatmap y del volumen de entrada
        heatmap_slice = heatmap.take(indices=i, axis=axis)
        input_slice = input_volume.take(indices=i, axis=axis)
        
        # Normalizar el heatmap entre 0 y 1 para visualizar
        heatmap_slice = np.maximum(heatmap_slice, 0)
        heatmap_slice /= np.max(heatmap_slice) if np.max(heatmap_slice) != 0 else 1

        # Mostrar el input_volume en la columna izquierda
        axs[2*i, 0].imshow(input_slice, cmap='gray')
        axs[2*i, 0].set_title(f'Input Slice {i} (axis {axis})')
        axs[2*i, 0].axis('off')  # Ocultar ejes para una visualización más limpia

        # Mostrar el heatmap en la columna derecha
        axs[2*i, 1].imshow(heatmap_slice, cmap='jet')
        axs[2*i, 1].set_title(f'Heatmap Slice {i} (axis {axis})')
        axs[2*i, 1].axis('off')  # Ocultar ejes para una visualización más limpia

    plt.tight_layout()
    plt.show()


In [ ]:
fig, axs = plt.subplots(12, 2, figsize=(12, 24))
for i in range(6):
    axs[i,0].imshow(heatmap[i], cmap='jet_r')
    axs[i,0].axis('off')
    axs[i,1].imshow(input_volume[i], cmap='jet_r') 
    axs[i,0].axis('off')
plt.show()

In [38]:
# Obtener los pesos de la capa de salida
output_weights = model.get_layer("conv3d_26").get_weights()[0]  # [0] para los pesos


In [18]:
# Redimensionar las activaciones si es necesario
cam = np.dot(activations[0], output_weights)

# Si es necesario, puedes aplicar una función ReLU
cam = np.maximum(cam, 0)

# Normalizar para hacer más visibles las diferencias
cam = (cam - cam.min()) / (cam.max() - cam.min())


ValueError: shapes (6,8,8,8) and (16,1) not aligned: 8 (dim 3) != 16 (dim 0)

In [12]:
import matplotlib.pyplot as plt

# Visualiza el CAM
plt.imshow(cam, cmap='jet')
plt.show()


NameError: name 'cam' is not defined

In [13]:
b = model.predict(a)
b

array([[0.99776864]], dtype=float32)

In [14]:
import numpy as np

# Usa una imagen de entrada con el mismo formato que tu modelo espera
# Por ejemplo: input_data = np.random.random((1, 6, 8, 8, 3))
activations = cam_model.predict(input_data)


NameError: name 'cam_model' is not defined

In [15]:
def get_grad_cam_3d(model, input_data, layer_name, class_idx):
    grad_model = models.Model(
        [model.inputs], 
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(input_data)
        loss = predictions[:, class_idx]

    # Gradientes de la salida con respecto a la capa convolucional
    grads = tape.gradient(loss, conv_outputs)
    
    # Pondera los gradientes por la importancia de la clase
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2, 3))

    # Multiplica cada canal por la importancia de los gradientes
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    # Aplica ReLU
    heatmap = np.maximum(heatmap, 0)
    
    # Normaliza entre 0 y 1
    heatmap /= np.max(heatmap)
    
    return heatmap#.numpy()


In [16]:
import tensorflow as tf

# Crear un modelo intermedio que devuelva las salidas del TimeDistributed
#intermediate_model = tf.keras.Model(inputs=model.inputs, 
#                                    outputs=model.get_layer('time_distributed_2').output)


In [17]:
p =  {'port': 8080, 'umbral': 0.51, 'sizeMax': 1000000.0, 'save': True, 'domain': [-88.0, -63.0, -25.0, 5.0], 'saveMax': 10000001, 'dirModelos': 'C:\\Users\\Shounen\\Desktop\\Ciclo XI\\Tesis 2\\SistemaQC/Modelos/', 'fecha': '2022-02-01-13-00', 'dato': 40.2, 'canales': ['07', '08', '13'], 'tiempos': ['00', '50', '40','30','20','10'], 'margen': 8, 'dibujar': False, 'canalDibujar': '13', 'hard_save': False, 'coordLon': -80.11716, 'coordLat': -4.71357, 'codigo': 'X47E0E1A2', 'ID': 1726442740}
fileName = r'C:\Users\Shounen\Desktop\Ciclo XI\Tesis 2\SistemaQC\Imagenes\2022-02-01-13-00.nc'

In [18]:
imagen, errors = mergeImagesFile(fileName, p)

In [19]:
imagen, errors = mergeImagesFile(fileName, p)
a = {
     inputLayers[0]: np.full((1, imagen.shape[0], imagen.shape[1],imagen.shape[2], imagen.shape[3]),imagen),
     inputLayers[1]: np.full((1,), 0),  # Precipitacion
     inputLayers[3]: np.full((1,), 1100),  # Altura
     inputLayers[2]: np.full((1,), 3)  # Umbral
    }

In [20]:
model.predict(a)

ValueError: in user code:

    File "C:\Users\Shounen\anaconda3\envs\WB-PY39\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Shounen\anaconda3\envs\WB-PY39\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Shounen\anaconda3\envs\WB-PY39\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Shounen\anaconda3\envs\WB-PY39\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\Shounen\anaconda3\envs\WB-PY39\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Shounen\anaconda3\envs\WB-PY39\lib\site-packages\keras\engine\input_spec.p

In [21]:
def get_grad_cam_3d(model, input_data, layer_name, class_idx):
    grad_model = models.Model(
        [model.inputs], 
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(input_data)
        loss = predictions[:, class_idx]

    # Gradientes de la salida con respecto a la capa convolucional
    grads = tape.gradient(loss, conv_outputs)
    
    return grads
    # Pondera los gradientes por la importancia de la clase
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2, 3))

    # Multiplica cada canal por la importancia de los gradientes
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    # Aplica ReLU
    heatmap = np.maximum(heatmap, 0)
    
    # Normaliza entre 0 y 1
    heatmap /= np.max(heatmap)
    
    return heatmap#.numpy()


In [22]:
heatmap = get_grad_cam_3d(model, a, "conv3d_26", class_idx=0)
heatmap.numpy().round(2)

ValueError: No such layer: conv3d_26. Existing layers are: ['input_22', 'rescaling_6', 'conv3d_36', 'batch_normalization_36', 'leaky_re_lu_36', 'conv3d_37', 'batch_normalization_37', 'leaky_re_lu_37', 'dropout_45', 'conv3d_38', 'batch_normalization_38', 'leaky_re_lu_38', 'dropout_46', 'conv3d_39', 'batch_normalization_39', 'leaky_re_lu_39', 'dropout_47', 'conv3d_40', 'batch_normalization_40', 'leaky_re_lu_40', 'dropout_48', 'conv3d_41', 'batch_normalization_41', 'leaky_re_lu_41', 'dropout_49', 'conv3d_42', 'batch_normalization_42', 'leaky_re_lu_42', 'dropout_50', 'global_max_pooling3d_5', 'input_23', 'input_24', 'input_25', 'concatenate_5', 'dense_25', 'dropout_51', 'dense_26', 'dropout_52', 'dense_27', 'dropout_53', 'dense_28', 'dense_29'].

In [23]:
heatmap = get_grad_cam_3d(model, a, "conv3d_26", class_idx=0)

# Visualizar la activación
plt.imshow(heatmap[:,:,0], cmap='viridis')
plt.colorbar()
plt.show()

ValueError: No such layer: conv3d_26. Existing layers are: ['input_22', 'rescaling_6', 'conv3d_36', 'batch_normalization_36', 'leaky_re_lu_36', 'conv3d_37', 'batch_normalization_37', 'leaky_re_lu_37', 'dropout_45', 'conv3d_38', 'batch_normalization_38', 'leaky_re_lu_38', 'dropout_46', 'conv3d_39', 'batch_normalization_39', 'leaky_re_lu_39', 'dropout_47', 'conv3d_40', 'batch_normalization_40', 'leaky_re_lu_40', 'dropout_48', 'conv3d_41', 'batch_normalization_41', 'leaky_re_lu_41', 'dropout_49', 'conv3d_42', 'batch_normalization_42', 'leaky_re_lu_42', 'dropout_50', 'global_max_pooling3d_5', 'input_23', 'input_24', 'input_25', 'concatenate_5', 'dense_25', 'dropout_51', 'dense_26', 'dropout_52', 'dense_27', 'dropout_53', 'dense_28', 'dense_29'].

In [24]:
heatmap = get_grad_cam_3d(model, a, "conv3d_26", class_idx=0)

# Visualizar la activación
plt.imshow(heatmap[:,:,0], cmap='viridis')
plt.colorbar()
plt.show()

ValueError: No such layer: conv3d_26. Existing layers are: ['input_22', 'rescaling_6', 'conv3d_36', 'batch_normalization_36', 'leaky_re_lu_36', 'conv3d_37', 'batch_normalization_37', 'leaky_re_lu_37', 'dropout_45', 'conv3d_38', 'batch_normalization_38', 'leaky_re_lu_38', 'dropout_46', 'conv3d_39', 'batch_normalization_39', 'leaky_re_lu_39', 'dropout_47', 'conv3d_40', 'batch_normalization_40', 'leaky_re_lu_40', 'dropout_48', 'conv3d_41', 'batch_normalization_41', 'leaky_re_lu_41', 'dropout_49', 'conv3d_42', 'batch_normalization_42', 'leaky_re_lu_42', 'dropout_50', 'global_max_pooling3d_5', 'input_23', 'input_24', 'input_25', 'concatenate_5', 'dense_25', 'dropout_51', 'dense_26', 'dropout_52', 'dense_27', 'dropout_53', 'dense_28', 'dense_29'].

In [25]:
heatmap

NameError: name 'heatmap' is not defined

In [26]:
input_data = [X_3d, X_input_60, X_input_61, X_input_62]  # Asegúrate de tener los datos en el formato correcto
heatmap = get_grad_cam_3d(model_14, input_data, "conv3d_4", class_idx=0)

# Visualizar la activación
plt.imshow(heatmap[0, :, :, :], cmap='viridis')
plt.colorbar()
plt.show()


NameError: name 'X_3d' is not defined

In [67]:
conv_outputs = intermediate_model.predict(a)

In [68]:
conv_outputs#.shape

array([[[ 96297.33  ,      0.    ,  98873.414 ,      0.    ,
              0.    ,  29097.32  ,  85643.41  ,  89785.36  ,
          86087.91  , 259413.27  , 238617.5   ,      0.    ,
              0.    , 302912.16  , 105283.56  ,      0.    ,
              0.    , 317176.1   ,      0.    ,  12163.904 ,
           8157.2256,   3226.119 ,  12670.005 , 135182.88  ,
              0.    ,      0.    , 531784.4   , 340744.97  ,
              0.    , 241008.03  ,      0.    , 128938.29  ,
         276576.72  ,      0.    ,  21678.344 ,      0.    ,
              0.    , 113671.234 ,      0.    ,      0.    ,
         142985.56  , 159298.66  ,  88692.516 , 170437.08  ,
              0.    ,  43530.805 ,      0.    ,  20896.24  ,
         135795.34  , 247485.44  ,      0.    ,      0.    ,
          48391.477 , 238829.16  ,      0.    ,      0.    ,
         268377.44  ,      0.    ,      0.    ,   9348.777 ,
              0.    ,  36776.375 ,  16014.831 , 408584.56  ],
        [ 96264.3   ,  

In [29]:
#model.predict(a)

array([[0.5847513]], dtype=float32)

In [44]:
model.get_layer('time_distributed_2').output

<KerasTensor: shape=(None, 3, 64) dtype=float32 (created by layer 'time_distributed_2')>

In [51]:
conv_model.get_layer('global_max_pooling2d_1').output

<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'global_max_pooling2d_1')>

In [58]:
import tensorflow as tf

# Asumiendo que 'model' es tu modelo original y 'conv_model' es tu modelo CNN dentro de TimeDistributed
def create_intermediate_model(model, layer_name):
    # Obtener las salidas de la capa deseada
    intermediate_layer_model = tf.keras.Model(inputs=model.input, 
                                              outputs=model.get_layer(layer_name).output)
    return intermediate_layer_model


In [61]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# Asumiendo que 'model' es tu modelo principal y 'conv_model' es tu modelo intermedio
# 1. Crear el modelo intermedio
conv_model = Model(inputs=model.input, outputs=model.get_layer('global_max_pooling2d_1').output)

# 2. Preparar la imagen para la inferencia
img_array = np.array([your_image_data])  # Asegúrate de que esto tenga el formato correcto
img_array = tf.convert_to_tensor(img_array, dtype=tf.float32)

# 3. Generar las activaciones y predicciones
with tf.GradientTape() as tape:
    # Obtener las activaciones
    conv_outputs = conv_model(img_array)
    tape.watch(conv_outputs)
    
    # Obtener la predicción de la clase
    predictions = model(img_array)
    class_id = tf.argmax(predictions[0]).numpy()
    
    # Obtener el gradiente para la clase predicha
    class_output = predictions[0][0, class_id]
    grads = tape.gradient(class_output, conv_outputs)

# 4. Calcular el CAM
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)

# Normalizar y mostrar el heatmap
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

# Puedes usar matplotlib para visualizar el heatmap
import matplotlib.pyplot as plt

plt.imshow(heatmap.squeeze(), cmap='jet')
plt.colorbar()
plt.show()


ValueError: No such layer: global_max_pooling2d_1. Existing layers are: ['input_5', 'time_distributed_2', 'time_distributed_3', 'lstm_1', 'input_6', 'input_7', 'input_8', 'concatenate_1', 'dense_3', 'dense_4', 'dense_5'].

In [59]:
# Convertir tus arrays de entrada a tf.Tensor
data = a
img_array = tf.convert_to_tensor(data['input_5'], dtype=tf.float32)
additional_inputs = [tf.convert_to_tensor(data['input_6'], dtype=tf.float32),
                     tf.convert_to_tensor(data['input_7'], dtype=tf.float32),
                     tf.convert_to_tensor(data['input_8'], dtype=tf.float32)]

# Ajusta el tamaño del batch si es necesario
img_array = tf.expand_dims(img_array, axis=0)  # Añade una dimensión de batch


In [60]:
# Crear el modelo intermedio
intermediate_model = create_intermediate_model(model, 'global_max_pooling2d_1')

# Obtén las activaciones
with tf.GradientTape() as tape:
    tape.watch(img_array)
    conv_outputs = intermediate_model(img_array)

# Calcula el gradiente
grads = tape.gradient(conv_outputs, img_array)

# Promedia el gradiente sobre las dimensiones del canal
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

# Multiplica los pesos de la activación
conv_outputs = conv_outputs[0]
for i in range(pooled_grads.shape[-1]):
    conv_outputs[:, :, :, i] *= pooled_grads[i]

# Calcula el mapa de activación
heatmap = tf.reduce_mean(conv_outputs, axis=-1)
heatmap = tf.maximum(heatmap, 0)
heatmap /= tf.max(heatmap)


ValueError: No such layer: global_max_pooling2d_1. Existing layers are: ['input_5', 'time_distributed_2', 'time_distributed_3', 'lstm_1', 'input_6', 'input_7', 'input_8', 'concatenate_1', 'dense_3', 'dense_4', 'dense_5'].